### Semana 5, Día 4

AutoGen Core - Distribuido

¡Solo les daré un adelanto!

En parte porque no estoy seguro de su relevancia. Si desean que agregue más contenido, por favor, háganmelo saber.

In [1]:
from dataclasses import dataclass
from autogen_core import AgentId, MessageContext, RoutedAgent, message_handler
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.messages import TextMessage
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_ext.tools.langchain import LangChainToolAdapter
from langchain_community.utilities import GoogleSerperAPIWrapper
from langchain.agents import Tool
from IPython.display import display, Markdown

from dotenv import load_dotenv

load_dotenv(override=True)

ALL_IN_ONE_WORKER = False

### Comienza con nuestra clase de Mensaje

In [2]:

@dataclass
class Message:
    content: str

### Y ahora: un host para nuestro entorno de ejecución distribuido

In [3]:
from autogen_ext.runtimes.grpc import GrpcWorkerAgentRuntimeHost

host = GrpcWorkerAgentRuntimeHost(address="localhost:50051")
host.start() 

### Vamos a reintroducir una herramienta

In [ ]:
serper = GoogleSerperAPIWrapper()
langchain_serper =Tool(name="internet_search", func=serper.run, description="Útil para cuando necesitas buscar en Internet.")
autogen_serper = LangChainToolAdapter(langchain_serper)

In [ ]:
instruction1 = "Para ayudar con una decisión sobre si usar AutoGen en un nuevo proyecto de Agentes de IA, \
por favor investiga y responde brevemente con razones a favor de elegir AutoGen; los pros de AutoGen."

instruction2 = "Para ayudar con una decisión sobre si usar AutoGen en un nuevo proyecto de Agentes de IA, \
por favor investiga y responde brevemente con razones en contra de elegir AutoGen; los contras de Autogen."

judge = "Debes tomar una decisión sobre si usar AutoGen para un proyecto. \
Tu equipo de investigación ha llegado a las siguientes razones a favor y en contra. \
Basado puramente en la investigación de tu equipo, por favor responde con tu decisión y una breve justificación."

### Y hacer algunos Agentes

In [ ]:
class Player1Agent(RoutedAgent):
    def __init__(self, name: str) -> None:
        super().__init__(name)
        model_client = OpenAIChatCompletionClient(model="gpt-4o-mini")
        self._delegate = AssistantAgent(name, model_client=model_client, tools=[autogen_serper], reflect_on_tool_use=True)

    @message_handler
    async def handle_my_message_type(self, message: Message, ctx: MessageContext) -> Message:
        text_message = TextMessage(content=message.content, source="user")
        response = await self._delegate.on_messages([text_message], ctx.cancellation_token)
        return Message(content=response.chat_message.content)
    
class Player2Agent(RoutedAgent):
    def __init__(self, name: str) -> None:
        super().__init__(name)
        model_client = OpenAIChatCompletionClient(model="gpt-4o-mini")
        self._delegate = AssistantAgent(name, model_client=model_client, tools=[autogen_serper], reflect_on_tool_use=True)

    @message_handler
    async def handle_my_message_type(self, message: Message, ctx: MessageContext) -> Message:
        text_message = TextMessage(content=message.content, source="user")
        response = await self._delegate.on_messages([text_message], ctx.cancellation_token)
        return Message(content=response.chat_message.content)
    
class Judge(RoutedAgent):
    def __init__(self, name: str) -> None:
        super().__init__(name)
        model_client = OpenAIChatCompletionClient(model="gpt-4o-mini")
        self._delegate = AssistantAgent(name, model_client=model_client)
        
    @message_handler
    async def handle_my_message_type(self, message: Message, ctx: MessageContext) -> Message:
        message1 = Message(content=instruction1)
        message2 = Message(content=instruction2)
        inner_1 = AgentId("player1", "default")
        inner_2 = AgentId("player2", "default")
        response1 = await self.send_message(message1, inner_1)
        response2 = await self.send_message(message2, inner_2)
        result = f"## Pros de AutoGen:\n{response1.content}\n\n## Cons de AutoGen:\n{response2.content}\n\n"
        judgement = f"{judge}\n{result}Responde con tu decisión y una breve justificación"
        message = TextMessage(content=judgement, source="user")
        response = await self._delegate.on_messages([message], ctx.cancellation_token)
        return Message(content=result + "\n\n## Decisión:\n\n" + response.chat_message.content)


In [7]:
from autogen_ext.runtimes.grpc import GrpcWorkerAgentRuntime

if ALL_IN_ONE_WORKER:

    worker = GrpcWorkerAgentRuntime(host_address="localhost:50051")
    await worker.start()

    await Player1Agent.register(worker, "player1", lambda: Player1Agent("player1"))
    await Player2Agent.register(worker, "player2", lambda: Player2Agent("player2"))
    await Judge.register(worker, "judge", lambda: Judge("judge"))

    agent_id = AgentId("judge", "default")

else:

    worker1 = GrpcWorkerAgentRuntime(host_address="localhost:50051")
    await worker1.start()
    await Player1Agent.register(worker1, "player1", lambda: Player1Agent("player1"))

    worker2 = GrpcWorkerAgentRuntime(host_address="localhost:50051")
    await worker2.start()
    await Player2Agent.register(worker2, "player2", lambda: Player2Agent("player2"))

    worker = GrpcWorkerAgentRuntime(host_address="localhost:50051")
    await worker.start()
    await Judge.register(worker, "judge", lambda: Judge("judge"))
    agent_id = AgentId("judge", "default")




In [8]:
response = await worker.send_message(Message(content="Go!"), agent_id)

In [ ]:
display(Markdown(response.content))

In [10]:
await worker.stop()
if not ALL_IN_ONE_WORKER:
    await worker1.stop()
    await worker2.stop()

In [11]:
await host.stop()